In [438]:
using Revise
using HypergraphNB
using DataFrames
using Clustering
using Statistics
using RCall
using Arpack
using SparseArrays
using Kronecker
using StatsBase

In [398]:
n  = 1000
c₂ = 10
c₃ = 10
z  = 1 .+ (1:n .> n/2);
p₂ = 0.7
p₃ = 0.7

0.7

In [399]:
H = detectabilityData(n, c₂, c₃, p₂, p₃);

In [400]:
function get_eig(H, z, reduced)
    BP_mat, ix = linearizedBPMatrix(H, z; reduced = reduced);
    E = Arpack.eigs(BP_mat; nev = 2, ritzvec = true)
    return E[2][:,2]
end

get_eig (generic function with 3 methods)

In [440]:
@time B, ix = reducedNonBacktrackingMatrices(H);
@time c, C = degreeTensor(H, z);

ẑ = z

n = length(ẑ)
@time q = 1/n * counts(z)


T = zero(C)
for k ∈ 1:size(C)[1]
    T[k,:,:] = (C[k,:,:] / ((k - 1) * c[k]) .- 1) .* q
end

@time not_nan = [k for k ∈ 1:size(T)[1] if !isnan(T[k,1,1])]

@time BP_mat = sum(Kronecker.kronecker(T[k,:,:], Bs[k]) for k ∈ not_nan);

  0.030484 seconds (231.66 k allocations: 14.229 MiB)
  0.022086 seconds (291.24 k allocations: 8.187 MiB)
  0.000012 seconds (3 allocations: 208 bytes)
  0.051734 seconds (67.69 k allocations: 3.321 MiB)
  0.209176 seconds (57.97 k allocations: 32.454 MiB, 3.94% gc time)


In [410]:
@time BP_mat, ix = linearizedBPMatrix(H, z; reduced = true);
@time E = Arpack.eigs(BP_mat; nev = 2, ritzvec = true); 

  0.979698 seconds (522.93 k allocations: 144.497 MiB, 13.38% gc time)
  0.151901 seconds (302 allocations: 1.304 MiB)


In [403]:
@time BP_mat, ix = linearizedBPMatrix(H, z; reduced = false);
@time E = Arpack.eigs(BP_mat; nev = 2, ritzvec = true); 

 77.195881 seconds (2.83 M allocations: 11.991 GiB, 0.15% gc time)
 14.942768 seconds (302 allocations: 12.815 MiB)


In [330]:
@time get_eig(H, z, true);
@time get_eig(H, z, false);

  0.013594 seconds (21.96 k allocations: 2.428 MiB)
  0.217122 seconds (47.90 k allocations: 33.133 MiB)


In [331]:
@time B̂s = reducedNonBacktrackingMatrices(H);
@time EV_  = Arpack.eigs(B̂s[3]; nev = 2, ritzvec = true);

  0.000863 seconds (9.40 k allocations: 685.875 KiB)
  0.001048 seconds (359 allocations: 103.219 KiB)


In [332]:
@time Bs, ix = nonBacktrackingMatrices(H; return_indices = true);
@time EV = Arpack.eigs(Bs[3]; nev = 2, ritzvec = true);

  0.011495 seconds (35.52 k allocations: 2.710 MiB)
  0.004640 seconds (376 allocations: 432.109 KiB)


In [288]:
EV_[1]

2-element Array{Complex{Float64},1}:
 10.108651583616924 + 0.0im
  6.157064385998919 + 0.0im

In [289]:

EV[1]

2-element Array{Complex{Float64},1}:
 10.10865158361692 + 0.0im
 6.157064385998914 + 0.0im